In [1]:
import io
import ftplib
import pandas as pd
import config
import json
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_chave_notas():
    today = date.today().strftime('%d-%m-%Y')
    yesterday = (date.today() - timedelta(1)).strftime('%Y-%m-%d')
    file_bytes = io.BytesIO()
    ocorrencias = ['114', '113']

    conecta_ftp = ftplib.FTP(config.HOSTNAME_CONECTA, config.USERNAME_CONECTA, config.PASSWORD_CONECTA)
    conecta_ftp.cwd(config.PATH_TRIMESTRAL)
    list_files = conecta_ftp.nlst()

    for file in list_files:
        if not today in file: continue
        conecta_ftp.retrbinary('RETR ' + file, file_bytes.write)
        df = pd.read_excel(file_bytes)
    conecta_ftp.close()

    df.loc[:,"Pagador do frete/Nome Fantasia"] = df["Pagador do frete/Nome Fantasia"].str.contains("MAGAZINE")
    df = df.drop_duplicates(subset=["Nota Fiscal"])
    df = df[df["Código Ocorrência"].astype(str).isin(ocorrencias)]
    df.loc[:,"Data da ocorrência"] = df["Data da ocorrência"].astype(str).str.split().str[0]
    df = df[df["Data da ocorrência"] == yesterday]
    chave_nota = df["Chave NF-e"].to_list()
    return chave_nota

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.maximize_window()
driver.get('https://lite.arquivei.com.br/batch')

login = driver.find_element(By.XPATH, '//*[@id="email"]')
login.send_keys('daniel@conectacargo.com.br')

password = driver.find_element(By.XPATH, '//*[@id="password"]')
password.send_keys('conecta123')

iframe = driver.find_element(By.XPATH, '//*[@id="google-recaptcha"]/div/div/iframe')
driver.switch_to.frame(iframe)

recaptcha = driver.find_element(By.XPATH, '//*[@id="recaptcha-anchor"]/div[1]')
recaptcha.click()
